<a href="https://colab.research.google.com/github/RudiksChess/UVG-DataScience-Notas-6-Semestre/blob/main/Clases/Clase%2013%20-%2014%20/Red_Neuronal_Basica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión Lineal Simple. Ejemplo minimalista

### Importar las librerías relevantes

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para graficar en 3-D
import time
import seaborn as sns

In [2]:
start = time.time()

### Generar datos al azar para entrenar al modelo

Trabajaremos con dos variables de entrada, las x1 y x2 en nuestros ejemplos anteriores. Se generan al azar a partir de una distribución uniforme.

Se creará una matriz con estas dos variables.  La matriz X del modelo lineal y = x * w + b

In [3]:
# Por facilidad, declaramos una variable que indique el tamaño del conjunto 
#      de datos de entrenamiento.
observaciones = 1000
x1 = np.random.uniform(low=-10, high=10, size=(observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

entradas = np.column_stack((x1,x2))

# Verificar la forma de la matriz 
# Debiera ser n x k, donde n es el número de observaciones, y k es el número de variables.
print (entradas.shape)

(1000, 2)


### Generar las metas a las que debemos apuntar

Inventaremos una función f(x1, x2) = 2 * x1 - 3 * x2 + 5 + <ruido pequeño>.  El ruido es para hacerlo más realista.

Utilizaremos la metodología de ML, y veremos si el algoritmo la ha aprendido.  

In [4]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

targets = 13 * x1 +7 * x2 -12 + ruido

# Veamos las dimensiones. Deben ser n x m, donde m es el número de variables de salida.
print (targets.shape)

(1000, 1)


### Graficar los datos a usar para el entrenamiento

La idea es ver que haya una fuerte tendencia que nuestro modelo debe aprender a reproducir.


In [5]:
print(x1.shape)
print(x2.shape)
print(targets.shape)

(1000, 1)
(1000, 1)
(1000, 1)


In [6]:
"""fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x1N = x1.reshape(observaciones,)
x2N = x2.reshape(observaciones,)
targetsN = targets.reshape(observaciones,)


ax.scatter(x1N, x2N, targetsN )

plt.show()"""
"""
x1N = x1.reshape(observaciones,)
x2N = x2.reshape(observaciones,)
targetsN = targets.reshape(observaciones,)

fig = px.scatter_3d(x = x1N, y = x2N, z = targetsN)

fig.update_layout(
    width = 500,
    height = 500,)

fig.show()"""

'\nx1N = x1.reshape(observaciones,)\nx2N = x2.reshape(observaciones,)\ntargetsN = targets.reshape(observaciones,)\n\nfig = px.scatter_3d(x = x1N, y = x2N, z = targetsN)\n\nfig.update_layout(\n    width = 500,\n    height = 500,)\n\nfig.show()'

### Inicializar variables

Inicializaremos los pesos y sesgos, al azar, dentre de un rango inicial pequeño.  Es posible "jugar" con este valor pero no es recomendable ya que el uso de rangos iniciales altos inhibe el aprendizaje por parte del algoritmo

Los pesos son de dimensiones k x m, donde k es el numero de variables de entrada y m es el número de variables de salida.  

Como solo hay una salida, el sesgo es de tamaño 1, y es un escalar

In [7]:
rango_inicial = 0.1

weights = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=(2, 1))

biases = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=1)

#Veamos cómo fueron inicializados.
print (weights)
print (biases)

[[-0.06726784]
 [ 0.0810325 ]]
[0.06678484]


In [8]:
weights.shape

(2, 1)

### Asignar la tasa de aprendizaje (Eta)

Se asigna un a tasa de aprendizaje pequeña.  Para este ejemplo funciona bien 0.02.  Vale la pena "jugar" con este valor para ver los resultados de hacerlo.

In [9]:
eta = 0.001

### Entrenar el modelo

Usaremos un valor de 100 para iterar sobre el conjunto de datos de entrenamiento.  Ese valor funciona bastante bien con la tasa de aprendizaje de 0.02.  Cómo saber el número adecuado de iteraciones es algo que veremos en futuras sesiones, pero generalmente una tasa de aprendizaje baja requiere de más iteraciones que una más alta.  Sin embargo hay que tener en mente que una tasa de aprendizaje alta puede causar que la pérdida "Loss" diverja a infinito, en vez de converger a cero (0)

Usaremos la función de pérdida L2-norm, pero dividido por 2, Para se consistente con la clase.  Es más, también lo dividiremos por el número de observaciones para obtener un promedio de pérdida por observación.  Hablamos en clase sobre la posiilidad de modificar esta función una vez no se pierda la característica de ser más baja para los resultados mejores, y vice versa.

Imprimimos la función de pérdida (loss) en cada iteración, para ver si está decreciendo como se desea.

Otro pequeño truco es escalar las deltas de la misma manera que se hizo con la función de pérdida.  De esta forma la tasa de aprendizaje es independiente del número de muestras (samples u observaciones).  De nuevo esto no cambia el principio, solo hace más fácil la selección de una tasa única de aprendizaje. 

Finalmente aplicamos la regla de actualización del decenso de gradiente.

Ojo!  los pesos son 2x1, la tasa de aprendizaje es 1x1 (escalar), las entradas son 1000x2, y las deltas escaladas son 1000x1.  Necesitamos obtener la transpuesta de las entradas para que no hayan problemas de dimensión en las operaciones. 



In [10]:
for i in range (100):
    
    # Esta es la ecuacion del modelo lineal: y = xw + b 
    salidas = np.dot(entradas, weights) + biases
    
    # Las deltas son las diferencias entre las salidas y las metas (targets)
    # deltas es un vector 1000 x 1
    deltas = salidas - targets
        
    loss = np.sum(deltas**2)/2  / observaciones
    
    print(loss)
    
    deltas_escaladas = deltas / observaciones
      
    weights = weights - eta * np.dot(entradas.T, deltas_escaladas)
    biases = biases - eta * np.sum(deltas_escaladas)
    
    # Los pesos son actualizados en una forma de algebra lineal(una matriz menos otra)
    # Sin embargo, los sesgos en este caso son solo un número, es necesario transformar las deltas
    #       a un escalar.      
    # Ambas lineas son consistentes con la metodología de decenso de gradiente-

3678.3950659338934
3443.8922328823205
3224.6238863739
3019.5997354479714
2827.8938626571803
2648.640539471969
2481.0303137054475
2324.3063512758267
2177.761015773109
2040.7326703714705
1912.6026876336664
1792.7926536933705
1680.7617541798395
1576.004330070687
1478.0475924265347
1386.4494856793824
1300.7966898179204
1220.7027524407506
1145.80634223543
1075.769615990017
1010.2766917569202
949.0322212686034
891.7600551532572
838.2019949179678
788.1166260590464
741.2782270258425
697.4757491071753
656.5118626300552
618.2020651600617
582.3738476729523
548.8659149290901
517.5274565272268
488.2174653432365
460.8041002735377
435.16409040318615
411.1821779058216
388.7505971577161
367.76858771182447
348.1419389307724
329.7825642207986
312.60810294244357
296.5415481988646
281.51089881960496
267.44883396699765
254.29240889462483
241.9827704828468
230.46489126579849
219.6873207478207
209.60195288543116
200.1638086839983
191.330832926593
183.06370411636047
175.3256567734711
168.0823152835464
161.3015

### Desplegamos los pesos y sesgos para ver si funcionaron correctamente.

Por el diseño de nuestro datos, los pesos debieran ser 2 y -3, y el sesgo: 5

**NOTA:**  Si aún no están los valores correctos, puede que aún estén convergiendo y sea necesario iterar más veces.  Para esto solo se requiere ejecutar la celda anterior cuantas veces sea requerido

In [11]:
print(weights, biases)      

[[12.55336282]
 [ 6.71983645]] [-1.0589199]


### Graficar las últimas salidas vrs las metas (targets)

Como son las últimas, luego del entrenamiento, representan el modelo final de exactitud.  Enntre más cercana esté esta gráfica a una línea de 45 grados, mhás cercanas están las salidas y metas.

Como este ejemplo es pequeño, es posible hacerlo, en los problemas que veremos más tarde en la clase, esto ya no sería posible.

In [12]:
"""salidasN = salidas.reshape(observaciones,)
targetsN = targets.reshape(observaciones,)
sns.scatterplot(x = salidasN, y =  targetsN)"""

"""
salidasN = salidas.reshape(observaciones,)
targetsN = targets.reshape(observaciones,)
fig = px.scatter(x = salidasN, y =  targetsN)

fig.update_layout(
    width = 400,
    height = 400,)

fig.show()"""

'\nsalidasN = salidas.reshape(observaciones,)\ntargetsN = targets.reshape(observaciones,)\nfig = px.scatter(x = salidasN, y =  targetsN)\n\nfig.update_layout(\n    width = 400,\n    height = 400,)\n\nfig.show()'

In [13]:
end = time.time()
print(end - start)

0.2210400104522705
